# <font color=red>Лекция 3.4</font> <font color=blue>Свойства и декораторы классов</font>

In [ ]:
from functools import lru_cache
@lru_cache
def fib(n):
    return fib(n-1)+fib(n-2) if n>1 else n
fib(100)

**Декораторы** Python весьма хороши, однако их достаточно сложно понять при первом знакомстве. **Декоратор в Python – это функция, которая принимает другую функцию в качестве аргумента.** Декоратор модифицирует или улучшает принятую функцию и выдает измененную. Это значит, что когда вы вызываете декорированную функцию, вы получите функцию, которая может иметь небольшие отличия, в виде дополнительных функций, совмещенных с базовым определением.

#### Пара примеров

Раз уж мы ознакомились со всеми аспектами функций в Python, давайте продемонстрируем их в коде:

In [ ]:
def hello_world():
    print('Hello world!')

**Мы можем хранить функции в переменных:**

In [ ]:
hello = hello_world
hello()

**Определять функции внутри других функций:**

In [ ]:
def wrapper_function():
    def hello_world():
        print('Hello world!')
    hello_world()
# hello_word() - ошибка
wrapper_function()

## Декораторы

Раз мы знаем, как работают функции, теперь мы можем понять как работают декораторы. Сначала посмотрим на пример декоратора:

In [ ]:
def decorator_function(func):
    def wrapper():
        print('Функция-обёртка!')
        print('Оборачиваемая функция: {}'.format(func))
        print('Выполняем обёрнутую функцию...')
        func()
        print('Выходим из обёртки')
    return wrapper

Здесь decorator_function() является функцией-декоратором. Как вы могли заметить, она является функцией высшего порядка, так как принимает функцию в качестве аргумента, а также возвращает функцию. Внутри decorator_function() мы определили другую функцию, обёртку, так сказать, которая обёртывает функцию-аргумент и затем изменяет её поведение. Декоратор возвращает эту обёртку. Теперь посмотрим на декоратор в действии:

In [ ]:
@decorator_function
def hello_world():
    print('Hello world!')
hello_world()

Просто добавив @decorator_function перед определением функции hello_world(), мы модифицировали её поведение. Однако как вы уже могли догадаться, выражение с @ является всего лишь синтаксическим сахаром для hello_world = decorator_function(hello_world).

Иными словами, выражение @decorator_function вызывает decorator_function() с hello_world в качестве аргумента и присваивает имени hello_world возвращаемую функцию.

Давайте взглянем на другие применения.
**Создание логируемого декоратора**
Возможно, вам потребуется логировать того, что делает ваша функция. Большую часть времени логинг будет встроен внутри вашей функции. Однако, бывают случаи, когда вам нужно сделать это на уровне функции, что бы получить представление о потоке программы или, возможно, для следования тем или иным условиям бизнеса, таким как аудит. Посмотрим на небольшой декоратор, который мы можем использовать для записи названия любой функции и того, что она делает:

In [ ]:
import logging

def log(func):
    # Логируем какая функция вызывается.
    def wrap_log(*args, **kwargs):
        name = func.__name__
        logger = logging.getLogger(name)
        logger.setLevel(logging.INFO)
    
        # Открываем файл логов для записи.
        fh = logging.FileHandler("%s.log" % name)
        fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        formatter = logging.Formatter(fmt)
        fh.setFormatter(formatter)
        logger.addHandler(fh)
        
        logger.info("Вызов функции: %s" % name)
        result = func(*args, **kwargs)
        logger.info("Результат: %s" % result)
        return func
    
    return wrap_log

@log
def double_function(a):
    # Умножаем полученный параметр
    return a*2


if __name__ == "__main__":
    value = double_function(2)

Этот небольшой скрипт содержит функцию log, которая принимает функцию как единственный аргумент. Мы создаем объект логгер, а название лог файла такое же, как и у функции. После этого, функция log будет записывать, как наша функция была вызвана и что она возвращает, если возвращает.

#### Встроенные декораторы

Python содержит несколько встроенных декораторов. Из всех этих декораторов, самой важной троицей являются:

    @classmethod
    @staticmethod
    @property

Также существуют декораторы в различных разделах стандартной библиотеки Python. Одним из примеров является functools.wraps. Мы сосредоточимся декораторе @property.

### Свойства Python (@property)

Python содержит очень удобный небольшой концепт, под названием property, который выполняет несколько полезных задач. Мы рассмотрим, как делать следующее:
- конвертация метода класс в атрибуты только для чтения;
- как реализовать сеттеры и геттеры в атрибут.

Один из самых простых способов использования property, это использовать его в качестве декоратора метода (). Это позволит вам превратить метод класса в атрибут класса. Это было очень полезно, когда нужно сделать какую-нибудь комбинацию значений.

Давайте взглянем на простой пример:

In [1]:
class Person(object):
    def __init__(self, first_name, last_name):
         #Конструктор
        self.first_name = first_name
        self.last_name = last_name
    
    @property
    def full_name(self):
         #Возвращаем полное имя
        return "%s %s" % (self.first_name, self.last_name)


В данном коде мы создали два класса атрибута, или свойств: self.first_name и self.last_name.
Далее мы создали метод full_name, который содержит декоратор <*@property>*. Это позволяет нам использовать следующий код в сессии интерпретатора:

In [2]:
person = Person("Mike", "Driscoll")
 
print(person.full_name) # Mike Driscoll
print(person.first_name) # Mike
 
person.full_name = "Jackalope"

Mike Driscoll
Mike


AttributeError: can't set attribute

Как вы видите, в результате превращение метода в свойство, мы можем получить к нему доступ при помощи обычной точечной нотации. Однако, если мы попытаемся настроить свойство на что-то другое, получим ошибку AttributeError. Единственный способ изменить свойство full_name, это сделать это косвенно: